
## 데이터셋 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import minmax_scale
from scipy.cluster.hierarchy import dendrogram
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
#merged csv 불러오기
path = "/content/drive/MyDrive/eda/merged.csv"
cols = ["가입자 일련번호", "성별코드", "연령대코드", "시도코드", "주상병코드", '요양일수', '입내원일수', '심결가산율', '심결요양급여비용총액', '심결본인부담금',
       '심결보험자부담금', '총처방일수']
df_raw = pd.read_csv(path, usecols = cols, encoding="utf-8")

In [ ]:
#데이터 확인
df_raw

,가입자 일련번호,성별코드,연령대코드,시도코드,주상병코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수
0,334176,2,16,41,I109,1,1,0.15,11540,1500,10040,30
1,334176,2,16,41,I109,1,1,0.15,11540,1500,10040,30
2,334209,2,14,41,I109,1,1,0.15,11540,1500,10040,60
3,334265,2,15,41,I109,1,1,0.15,11540,1500,10040,60
4,335323,2,14,41,I109,1,1,0.15,11540,1500,10040,30
...,...,...,...,...,...,...,...,...,...,...,...,...
11733708,995389,1,3,41,H5019,2,9,0.30,1191400,36540,1154860,0
11733709,996077,2,3,46,S925,2,3,0.25,836660,44450,792210,0
11733710,997217,1,3,41,G_,1,90,0.30,605110,60300,544810,0
11733711,997217,1,3,41,J852,11,101,0.30,4965320,308540,4656780,0


## 데이터 전처리

In [ ]:
""""
Data Preprocessing
----------------------------------------------------------------------------------------
feature : 원핫인코딩된 시도코드 (2), 원핫인코딩된 주상병코드 (22), '연령대코드', '요양일수', '입내원일수', '심결가산율', '심결요양급여비용총액', '심결본인부담금','심결보험자부담금', '총처방일수'
최종 인풋에서 제외되어야할 칼럼: ['가입자 일련번호', '주상병코드', '시도코드']
----------------------------------------------------------------------------------------
1. 주상병코드 : 가장 앞의 대분류만 남기기 -> 원핫인코딩
2. 시도코드 : 메타데이터 pdf의 정보를 활용, 광역시에 부여된 번호는 si로, 도에 부여된 번호는 do로 변환 -> 원핫인코딩
3. 성별코드 : 남자(1), 여자(2) -> 한번에 min, max normalization 시 자동으로 원핫인코딩 -> 남자(0), 여자(1)
"""


#데이터프레임 복사
temp = df_raw.copy(deep = True)

#메모리 free
del df_raw
gc.collect()

#["주상병코드"] 데이터 대분류로 축소
temp["주상병코드"] = temp["주상병코드"].str.slice(0,1)

#시는 si, 도는 do로 변환
temp["시도코드"] = temp["시도코드"].replace(
    {11 : "si"
    ,26 : "si"
    ,27 : "si"
    ,28 : "si"
    ,29 : "si"
    ,30 : "si"
    ,31 : "si"
    ,36 : "si"
    ,41 : "do"
    ,42 : "do"
    ,43 : "do"
    ,44 : "do"
    ,45 : "do"
    ,46 : "do"
    ,47 : "do"
    ,48 : "do"
    ,49 : "do"})

#주상병코드, 시도코드 원핫인코딩
temp = pd.get_dummies(temp, columns=["주상병코드", "시도코드"])

#원핫인코딩 확인
temp

,가입자 일련번호,성별코드,연령대코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,...,주상병코드_O,주상병코드_P,주상병코드_Q,주상병코드_R,주상병코드_S,주상병코드_T,주상병코드_U,주상병코드_Z,시도코드_do,시도코드_si
0,334176,2,16,1,1,0.15,11540,1500,10040,30,...,0,0,0,0,0,0,0,0,1,0
1,334176,2,16,1,1,0.15,11540,1500,10040,30,...,0,0,0,0,0,0,0,0,1,0
2,334209,2,14,1,1,0.15,11540,1500,10040,60,...,0,0,0,0,0,0,0,0,1,0
3,334265,2,15,1,1,0.15,11540,1500,10040,60,...,0,0,0,0,0,0,0,0,1,0
4,335323,2,14,1,1,0.15,11540,1500,10040,30,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11733708,995389,1,3,2,9,0.30,1191400,36540,1154860,0,...,0,0,0,0,0,0,0,0,1,0
11733709,996077,2,3,2,3,0.25,836660,44450,792210,0,...,0,0,0,0,1,0,0,0,1,0
11733710,997217,1,3,1,90,0.30,605110,60300,544810,0,...,0,0,0,0,0,0,0,0,1,0
11733711,997217,1,3,11,101,0.30,4965320,308540,4656780,0,...,0,0,0,0,0,0,0,0,1,0


## Normalization.
Min-max normalizaiton.

In [ ]:
#Normalization 해줘야하는 칼럼 리스트로 정리
norm_cols = ["성별코드", "연령대코드", "요양일수", "입내원일수", "심결가산율", "심결요양급여비용총액", "심결본인부담금", "심결보험자부담금", "총처방일수"]

#normalization 수행
for column in norm_cols:
  data = temp[column]
  temp[column] = minmax_scale(data)

#normalization 결과 확인
temp

,가입자 일련번호,성별코드,연령대코드,요양일수,입내원일수,심결가산율,심결요양급여비용총액,심결본인부담금,심결보험자부담금,총처방일수,...,주상병코드_O,주상병코드_P,주상병코드_Q,주상병코드_R,주상병코드_S,주상병코드_T,주상병코드_U,주상병코드_Z,시도코드_do,시도코드_si
0,334176,1.0,0.882353,0.004115,0.001190,0.500000,0.000055,0.000071,0.000050,0.033333,...,0,0,0,0,0,0,0,0,1,0
1,334176,1.0,0.882353,0.004115,0.001190,0.500000,0.000055,0.000071,0.000050,0.033333,...,0,0,0,0,0,0,0,0,1,0
2,334209,1.0,0.764706,0.004115,0.001190,0.500000,0.000055,0.000071,0.000050,0.066667,...,0,0,0,0,0,0,0,0,1,0
3,334265,1.0,0.823529,0.004115,0.001190,0.500000,0.000055,0.000071,0.000050,0.066667,...,0,0,0,0,0,0,0,0,1,0
4,335323,1.0,0.764706,0.004115,0.001190,0.500000,0.000055,0.000071,0.000050,0.033333,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11733708,995389,0.0,0.117647,0.008230,0.010714,1.000000,0.005665,0.001727,0.005785,0.000000,...,0,0,0,0,0,0,0,0,1,0
11733709,996077,1.0,0.117647,0.008230,0.003571,0.833333,0.003978,0.002101,0.003969,0.000000,...,0,0,0,0,1,0,0,0,1,0
11733710,997217,0.0,0.117647,0.004115,0.107143,1.000000,0.002877,0.002850,0.002729,0.000000,...,0,0,0,0,0,0,0,0,1,0
11733711,997217,0.0,0.117647,0.045267,0.120238,1.000000,0.023609,0.014582,0.023329,0.000000,...,0,0,0,0,0,0,0,0,1,0


## 필요없는 칼럼 제외 및 가입자 일련 번호 별도 저장, csv export

In [ ]:
#temp column 확인
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11733713 entries, 0 to 11733712
Data columns (total 34 columns):
 #   Column      Dtype  
---  ------      -----  
 0   가입자 일련번호    int64  
 1   성별코드        float64
 2   연령대코드       float64
 3   요양일수        float64
 4   입내원일수       float64
 5   심결가산율       float64
 6   심결요양급여비용총액  float64
 7   심결본인부담금     float64
 8   심결보험자부담금    float64
 9   총처방일수       float64
 10  주상병코드_A     uint8  
 11  주상병코드_B     uint8  
 12  주상병코드_C     uint8  
 13  주상병코드_D     uint8  
 14  주상병코드_E     uint8  
 15  주상병코드_F     uint8  
 16  주상병코드_G     uint8  
 17  주상병코드_H     uint8  
 18  주상병코드_I     uint8  
 19  주상병코드_J     uint8  
 20  주상병코드_K     uint8  
 21  주상병코드_L     uint8  
 22  주상병코드_M     uint8  
 23  주상병코드_N     uint8  
 24  주상병코드_O     uint8  
 25  주상병코드_P     uint8  
 26  주상병코드_Q     uint8  
 27  주상병코드_R     uint8  
 28  주상병코드_S     uint8  
 29  주상병코드_T     uint8  
 30  주상병코드_U     uint8  
 31  주상병코드_Z     uint8  
 32  시도코드_do     uint8  
 33  시도코드_

In [ ]:
#최종 데이터프레임
#클러스터링 때 마다 가입자 일련번호를 분리 저장할 것
temp.to_csv("/content/drive/MyDrive/eda/processed_data.csv", index = False)

print("전처리 끝!")

전처리 끝!
